# Silver To Gold Transformation (Level 2)

In this level of transformation data is usually curated based on business logic. Since this project just focuses on migrating the data no business logic is applied here. Instead modification of columns is taken place, where ***white spaces(" ")*** in column names are replaced by an ***underscore("_")***.

In [ ]:
dbutils.fs.ls("/mnt/silver/SalesLT")

In [ ]:
dbutils.fs.ls("/mnt/gold")

In [ ]:
table_names = []

for i in dbutils.fs.ls("/mnt/silver/SalesLT"):
    table_names.append(i.name.split('/')[0])

In [ ]:
from pyspark.sql.functions import col, regexp_replace

for table in table_names:
    table_path = "/mnt/silver/SalesLT/" + table
    df = spark.read.format("delta").load(table_path)
    columns = df.columns

    for old_column in columns:
        new_column = "".join(["_" + char if char.isupper() and not old_column[i-1].isupper() else char for i, char in enumerate(old_column)]).lstrip("_")
        df = df.withColumnRenamed(old_column, new_column)
    
    output_path = "/mnt/gold/SalesLT/" + table + "/"
    df.write.format("delta").mode("overwrite").save(output_path)